In [44]:
!pip install config # 주피터 노트북에서 해당 모듈을 설치할때는 pip 명령어를 사용 
# ex) !pip install *** or pip install ***

In [53]:
# 기본 셋팅, 사용할 import
import os
import sys
import urllib.request
import datetime
import pandas as pd
import time
import json
from config import *

import matplotlib.pyplot as plt
import matplotlib

In [54]:
#http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList
# 서비스 URL

In [55]:
# Python 샘플 코드 #


# import requests

# url = 'http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList'
# params ={'serviceKey' : '서비스키', 'YM' : '201201', 'NAT_CD' : '112', 'ED_CD' : 'D' }

# response = requests.get(url, params=params)
# print(response.content)

In [56]:
# [CODE 1]
def getRequestUrl(url):
    req = urllib.request.Request(url)
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8') 
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [86]:
#[CODE 2]
def getTourismStatsItem(yyyymm, nat_cd, ed_cd):
    service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"
#공공데이터에서 신청된 이후 제공하는 url 주소를 불러오기

    parameters = "?_type=json&serviceKey=" + "utjhR9RqKrAojKs8Bq57OxR2E28NLhynjLMOb4r%2FJPSB38yp8QAQoZNoB1EKnFNw9p17Kg%2BTwiYMvwCibLqCzg%3D%3D" #인증키
    parameters += "&YM=" + yyyymm
    parameters += "&NAT_CD=" + nat_cd
    parameters += "&ED_CD=" + ed_cd


    url = service_url + parameters

    responseDecode = getRequestUrl(url) # [CODE 1]

    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

In [87]:
#[CODE 3]
def getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear):
    jsonResult = []
    result = []
    natName = ""
    ed = []
    dataEND = []

 
    for year in range(nStartYear, nEndYear+1):
        for month in range(1, 13):
            yyyymm = "{0}{1:0>2}".format(str(year), str(month))
            jsonData = getTourismStatsItem(yyyymm, nat_cd, ed_cd) #[CODE 2]
            if (jsonData['response']['header']['resultMsg'] == 'OK'): 
                #데이터가 없는 마지막 항목인 경우 ----------------------------
                if jsonData['response']['body']['items'] == '':
                    dataEND = "{0}{1:0>2}".format(str(year), str(month-1))
                    print("데이터 없음.... \n 제공되는 통계 데이터는 %s년 %s월까지입니다." %(str(year), str(month-1)))
                    break
            #jsonData를 출력하여 확인...........................................
                print(json.dumps(jsonData, indent = 4, sort_keys = True, ensure_ascii = False))
                dataEND=yyyymm
                natName = jsonData['response']['body']['items']['item']["natKorNm"]
                natName = natName.replace(' ', '')
                num = jsonData['response']['body']['items']['item']['num']
                ed = jsonData['response']['body']['items']['item']['ed']
                print('[ %s_%s : %s ]' %(natName, yyyymm, num))
                print('-----------------------------------------------------')
                jsonResult.append({'nat_name': natName, 'nat_cd': nat_cd,'yyyymm': yyyymm, 'visit_cnt': num})
                result.append([natName, nat_cd, yyyymm, num])
    print(jsonData)
    return jsonResult, result, natName, ed, dataEND

In [88]:
def main():
    
    #중국: 112 / 일본: 130 / 미국: 275
    print("<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>")
    nat_cd = input('국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) :')
    nStartYear = int(input('데이터를 몇 년부터 수집할까요? : '))
    nEndYear = int(input('데이터를 몇 년까지 수집할까요? : '))
    ed_cd = "E" #E : 방한외래관광객, D : 해외 출국

    jsonResult, result, natName, ed, dataEND = getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear)  #[CODE 3]

    #파일저장 1 : json 파일
    with open('./%s_%s_해외방문객정보_%d_%s.json' % (natName, ed_cd, nStartYear, dataEND), 'w', encoding = 'utf8') as outfile:
        jsonFile = json.dumps(jsonResult, indent = 4, sort_keys = True, ensure_ascii = False)
        outfile.write(jsonFile)

    #파일저장 2 : csv 파일
    columns = ["입국자국가", "국가코드", "입국연월", "입국자 수"]
    result_df = pd.DataFrame(result, columns = columns)
    result_df.to_csv('./%s_%s_%d_%s.csv' % (natName, ed, nStartYear,dataEND), index=False, encoding='cp949')

In [89]:
main()

<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>
국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) :112
데이터를 몇 년부터 수집할까요? : 1990
데이터를 몇 년까지 수집할까요? : 2000
[2021-10-26 14:53:08.728234] Url Request Success
[2021-10-26 14:53:08.751429] Url Request Success
[2021-10-26 14:53:08.772721] Url Request Success
[2021-10-26 14:53:08.796275] Url Request Success
[2021-10-26 14:53:08.819051] Url Request Success
[2021-10-26 14:53:08.841053] Url Request Success
[2021-10-26 14:53:08.867523] Url Request Success
[2021-10-26 14:53:08.892666] Url Request Success
[2021-10-26 14:53:08.913626] Url Request Success
[2021-10-26 14:53:08.936342] Url Request Success
[2021-10-26 14:53:08.963966] Url Request Success
[2021-10-26 14:53:08.983484] Url Request Success
[2021-10-26 14:53:09.004063] Url Request Success
[2021-10-26 14:53:09.023692] Url Request Success
[2021-10-26 14:53:09.045731] Url Request Success
[2021-10-26 14:53:09.070233] Url Request Success
[2021-10-26 14:53:09.094240] Url Request Success
[2021-10-26 14:53:09.115719] Url Requ